# How to fine-tune open LLMs in 2025 with Hugging Face

Large Language Models (LLMs) continued their important role in 2024, with several major developments completely outperforming previous models. The focus continued to more smaller, more powerful models from companies like Meta, Qwen, or Google. These models not only became more powerful, but also more efficient. We got Llama models as small as 1B parameters outperforming Llama 2 13B. 

LLMs can now handle many tasks out-of-the-box through prompting, including chatbots, question answering, and summarization. However, for specialized applications requiring high accuracy or domain expertise, fine-tuning remains a powerful approach to achieve higher quality results than prompting alone, reduce costs by training smaller, more efficient models, and ensure reliability and consistency for specific use cases.

Contrary to last years guide [How to Fine-Tune LLMs in 2024 with Hugging Face](https://www.philschmid.de/fine-tune-llms-in-2024-with-trl) this guide focuses more on optimization, distributed training and being more customizable. This means support for different PEFT methods from Full-Finetuning to QLoRA and Spectrum, optimizations for faster and more efficient training, with Flash Attention or Tiger Kernels and how to scale training to multiple GPUs using DeepSpeed. 

This guide is created using a script rather than notebook. If you are compeltely new to fine-tuning LLMs, I recommend you to start with the [How to Fine-Tune LLMs in 2024 with Hugging Face](https://www.philschmid.de/fine-tune-llms-in-2024-with-trl) guide and then come back to this guide. 

You will learn how to:
1. Define a good use case for fine-tuning
2. Setup the development environment
3. Create and prepare the dataset
4. Fine-tune the model using `trl` and the `SFTTrainer`
5. Test and evaluate the model using `lighteval`
6. Deploy the model in to a production environment

_Note: This guide is designed for consumer GPUs (24GB+) like the NVIDIA RTX 4090/5090 or A10G, but can be adapted for larger systems._

## 1. Define a good use case for fine-tuning

Open LLMs became more powerful and smaller in 2024. This often could mean fine-tuning might not be the first choice to solve your problem. Before you think about fine-tuning, you should always evaluate if prompting or already fine-tuned models can solve your problem. Create an evaluation setup and compare the performance of existing open models. 

However, fine-tuning can be particularly valuable in several scenarios. When you need to:
- Consistently improve performance on a specific set of tasks
- Control the style and format of model outputs (e.g., enforcing a company's tone of voice)
- Teach the model domain-specific knowledge or terminology
- Reduce hallucinations for critical applications
- Optimize for latency by creating smaller, specialized models
- Ensure consistent adherence to specific guidelines or constraints

As an example, we are going to use the following use case:

> We want to fine-tune a model, which can solve high-school math problems to teach students how to solve math problems. 

This can be a good use case for fine-tuning, as it requires a lot of domain-specific knowledge about math and how to solve math problems. 

_Note: This is a made-up example, as existing open models already can solve this task._

## 2. Setup development environment

Our first step is to install Hugging Face Libraries and Pyroch, including trl, transformers and datasets. If you haven't heard of trl yet, don't worry. It is a new library on top of transformers and datasets, which makes it easier to fine-tune, rlhf, align open LLMs. 


In [ ]:
# Install Pytorch & other libraries
%pip install "torch==2.4.1" tensorboard flash-attn "liger-kernel==0.4.2" "setuptools<71.0.0" "deepspeed==0.15.4" openai

# Install Hugging Face libraries
%pip install  --upgrade \
  "transformers==4.46.3" \
  "datasets==3.1.0" \
  "accelerate==1.1.1" \
  "bitsandbytes==0.44.1" \
  "trl==0.12.1" \
  "peft==0.13.2" \
  "lighteval==0.6.2" \
  "hf-transfer==0.1.8"

We will use the [Hugging Face Hub](https://huggingface.co/models) as a remote model versioning service. This means we will automatically push our model, logs and information to the Hub during training. You must register on the [Hugging Face](https://huggingface.co/join) for this. After you have an account, we will use the `login` util from the `huggingface_hub` package to log into our account and store our token (access key) on the disk.



In [ ]:
from huggingface_hub import login

login(token="", add_to_git_credential=True) # ADD YOUR TOKEN HERE

## 3. Create and prepare the dataset

Once you've determined that fine-tuning is the right solution, you'll need a dataset. Most datasets are now created using automated synthetic workflows with LLMs, though several approaches exist:

* **Synthetic Generation with LLMs**: Most common approach using frameworks like [Distilabel](https://distilabel.argilla.io/) to generate high-quality synthetic data at scale
* **Existing Datasets**: Using public datasets from [Hugging Face Hub](https://huggingface.co/datasets)
* **Human Annotation**: For highest quality but most expensive option

The [LLM Datasets](https://github.com/mlabonne/llm-datasets) provides an overview of high-quality datasets to fine-tune LLMs for all kind of purposes. For our example, we'll use [Orca-Math](https://huggingface.co/datasets/microsoft/orca-math-word-problems-200k) dataset including 200,000 Math world problems. 

Modern fine-tuning frameworks like `trl` support standard formats:

```json
// Conversation format
{"messages": [
    {"role": "system", "content": "You are..."},
    {"role": "user", "content": "..."},
    {"role": "assistant", "content": "..."}
]}

// Instruction format 
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
```

_Note: If you are interested in a guide on how to create high-quality datasets, let me know._

To prepare our datasets we will use the Datasets library and then convert it into the the conversational format, where we include the schema definition in the system message for our assistant. We'll then save the dataset as jsonl file, which we can then use to fine-tune our model.

_Note: This step can be different for your use case. For example, if you have already a dataset from, e.g. working with OpenAI, you can skip this step and go directly to the fine-tuning step._

In [ ]:
from datasets import load_dataset

# Create system prompt
system_message = """Solve the given high school math problem by providing a clear explanation of each step leading to the final solution.

Provide a detailed breakdown of your calculations, beginning with an explanation of the problem and describing how you derive each formula, value, or conclusion. Use logical steps that build upon one another, to arrive at the final answer in a systematic manner.

# Steps

1. **Understand the Problem**: Restate the given math problem and clearly identify the main question and any important given values.
2. **Set Up**: Identify the key formulas or concepts that could help solve the problem (e.g., algebraic manipulation, geometry formulas, trigonometric identities).
3. **Solve Step-by-Step**: Iteratively progress through each step of the math problem, justifying why each consecutive operation brings you closer to the solution.
4. **Double Check**: If applicable, double check the work for accuracy and sense, and mention potential alternative approaches if any.
5. **Final Answer**: Provide the numerical or algebraic solution clearly, accompanied by appropriate units if relevant.

# Notes

- Always clearly define any variable or term used.
- Wherever applicable, include unit conversions or context to explain why each formula or step has been chosen.
- Assume the level of mathematics is suitable for high school, and avoid overly advanced math techniques unless they are common at that level.
"""

# convert to messages 
def create_conversation(sample):
  return {
    "messages": [
      {"role": "system", "content": system_message},
      {"role": "user", "content": sample["question"]},
      {"role": "assistant", "content": sample["answer"]}
    ]
  }  

# Load dataset from the hub
dataset = load_dataset("microsoft/orca-math-word-problems-200k", split="train")

# Convert dataset to OAI messages
dataset = dataset.map(create_conversation, remove_columns=dataset.features, batched=False)

print(dataset[345]["messages"])

# save datasets to disk 
dataset.to_json("train_dataset.json", orient="records")

## 4. Fine-tune LLM using `trl` and the `SFTTrainer` 

We are now ready to fine-tune our model. We will use the [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer) from `trl` to fine-tune our model. The `SFTTrainer` makes it straightfoward to supervise fine-tune open LLMs. The `SFTTrainer` is a subclass of the `Trainer` from the `transformers` library and supports all the same features, including logging, evaluation, and checkpointing, but adds additiional quality of life features, including:
* Dataset formatting, including conversational and instruction format
* Training on completions only, ignoring prompts
* Packing datasets for more efficient training
* PEFT (parameter-efficient fine-tuning) support including Q-LoRA, or Spectrum
* Preparing the model and tokenizer for conversational fine-tuning (e.g. adding special tokens)
* distributed training with `accelerate` and FSDP/DeepSpeed

We prepared a [run_sft.py](./scripts/run_sft.py) scripts, which supports providing a yaml configuration file to run the fine-tuning. This allows you to easily change the model, dataset, hyperparameters, and other settings. This is done by using the `TrlParser`, which parses the yaml file and converts it into the `TrainingArguments` arguments. That way we can support Q-LoRA, Spectrum, and other PEFT methods with the same script. See Appendix A for execution examples for different models and PEFT methods and distributed training.

> Question: Why don't we use frameworks like [axolotl](https://github.com/axolotl-ai-cloud/axolotl)? 

That's a good question! Axolotl is a great framework and I highly recommend it! However, it is always good to know how to do things manually. This will give you a better understanding of the inner workings of the framework and how it can be customized. Especially when you ran into an issue or want to extend the scripts and add new features. Axolotl is used by many open source builders and is well tested.

Before we can start our training lets take a look at our [training script](./scripts/run_sft.py). The script is kept very simple and is easy to understand. This should help you understand, customize and extend the script for your own use case. We define `dataclasses` for our arguments. Every argument can then be provided either via the command line or by providing a yaml configuration file. That way we have better type safety and intellisense support.

```python
# ....

@dataclass
class ScriptArguments:
    dataset_id_or_path: str
    ...
# ....
```

That way we can customize behavior for different training methods and use them in our script with `script_args.method`. The training script is separated by `#######` blocks that separate the different parts of the script. The main training function: 
1. Logs all parameters
2. Loads the dataset from Hugging Face Hub or local disk
3. Loads the tokenizer and model with our training strategy (e.g. Q-LoRA, Spectrum)
4. Initializes the `SFTTrainer`
5. Starts the training loop (optionally continue training from a checkpoint)
6. Saves the model and optionally pushes it to the Hugging Face Hub

Below is an example recipe of how we can fine-tune a [Llama-3.1-8B model with Q-LoRA](./receipes/llama-3-1-8b-qlora.yaml). 

```yaml
# Model arguments
model_name_or_path: Meta-Llama/Meta-Llama-3.1-8B
tokenizer_name_or_path: Meta-Llama/Meta-Llama-3.1-8B-Instruct
model_revision: main
torch_dtype: bfloat16
attn_implementation: flash_attention_2
use_liger: true
bf16: true
tf32: true

# Dataset arguments
dataset_id_or_path: train_dataset.json
max_seq_length: 1024
packing: true
per_device_train_batch_size: 16
gradient_accumulation_steps: 4
gradient_checkpointing: true
gradient_checkpointing_kwargs:
  use_reentrant: false
output_dir: data/llama-3-1-8b-math-orca-qlora

# LoRA arguments
use_peft: true
load_in_4bit: true
merge_adapter: false
lora_target_modules: "all-linear"
# important as we need to train the special tokens for the chat template of llama 
lora_modules_to_save: ["lm_head", "embed_tokens"] # you might need to change this for qwen or other models
lora_r: 16
lora_alpha: 16

# Training arguments
num_train_epochs: 1
learning_rate: 2.0e-4 
lr_scheduler_type: constant
warmup_ratio: 0.1

# Logging arguments
logging_strategy: steps
logging_steps: 5
report_to:
- tensorboard
save_strategy: "epoch"
seed: 42

# Hugging Face Hub 
push_to_hub: false
hub_model_id: llama-3-1-8b-math-orca-qlora
hub_strategy: every_save
```

This config works for single-GPU training and for multi-GPU training with FSDP (see Appendix for full command).

In [ ]:
!python scripts/run_sft.py --config receipes/llama-3-1-8b-qlora.yaml

I ran several experiments with different optimization strategies, including Flash Attention, Liger Kernels, Q-Lora, Spectrum, and Full-Finetuning to compare the time it takes to fine-tune a model. The results are summarized in the following table:
 
| Model | Train samples | Hardware | Method | train sequence length | per device batch size | gradient accumulation  | packing | Flash Attention | Liger Kernels | est. optimization steps | est. train time |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Llama-3.1-8B | 10,000 | 1x L4 24GB | Q-LoRA | 1024 | 1 | 2 | ❌ | ❌ | ❌ | 5000 | ~360 min |
| Llama-3.1-8B | 10,000 | 1x L4 24GB | Q-LoRA | 1024 | 2 | 2 | ✅ | ❌ | ❌ | 1352 | ~290 min |
| Llama-3.1-8B | 10,000 | 1x L4 24GB | Q-LoRA | 1024 | 2 | 4 | ✅ | ✅ | ❌ | 676 | ~220 min |
| Llama-3.1-8B | 10,000 | 1x L4 24GB | Q-LoRA | 1024 | 4 | 4 | ✅ | ✅ | ✅ | 338 | ~135 min |
| Llama-3.1-8B | 10,000 | 4x L4 24GB | Q-LoRA | 1024 | 8 | 2 | ✅ | ✅ | ✅ | 84 | ~33 min |
| Llama-3.1-8B | 10,000 | 8x L4 24GB | Q-LoRA | 1024 | 8 | 2 | ✅ | ✅ | ✅ | 21 | ~18 min |


**Notes:** 
- Q-Lora included training the embedding layer and the lm_head, as we use the Llama 3.1 chat template and in the base model the special tokens are not trained.
- For distributed training Deepspeed (0.15.4) with ZeRO3 and Hugging Face Accelerate was used.


## 5. Test Model and run Inference

After the training is done we want to evaluate and test our model. We will load different samples from the original dataset and evaluate the model on those samples, using a simple loop and accuracy as our metric. 

_Note: Evaluating Generative AI models is not a trivial task since 1 input can have multiple correct outputs. If you want to learn more about evaluating generative models, check out [Evaluate LLMs and RAG a practical example using Langchain and Hugging Face](https://www.philschmid.de/evaluate-llm) blog post._



In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline
 
peft_model_id = "data/llama-3-1-8b-math-orca-qlora"
 
# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
tok = pipe.tokenizer
model = pipe.model

question = "What is 2+2?"
# Create system prompt
system_message = """Solve the given high school math problem by providing a clear explanation of each step leading to the final solution.

Provide a detailed breakdown of your calculations, beginning with an explanation of the problem and describing how you derive each formula, value, or conclusion. Use logical steps that build upon one another, to arrive at the final answer in a systematic manner.

# Steps

1. **Understand the Problem**: Restate the given math problem and clearly identify the main question and any important given values.
2. **Set Up**: Identify the key formulas or concepts that could help solve the problem (e.g., algebraic manipulation, geometry formulas, trigonometric identities).
3. **Solve Step-by-Step**: Iteratively progress through each step of the math problem, justifying why each consecutive operation brings you closer to the solution.
4. **Double Check**: If applicable, double check the work for accuracy and sense, and mention potential alternative approaches if any.
5. **Final Answer**: Provide the numerical or algebraic solution clearly, accompanied by appropriate units if relevant.

# Notes

- Always clearly define any variable or term used.
- Wherever applicable, include unit conversions or context to explain why each formula or step has been chosen.
- Assume the level of mathematics is suitable for high school, and avoid overly advanced math techniques unless they are common at that level.
"""


prompt = tok.apply_chat_template([{"role": "system", "content": system_message}, {"role": "user", "content": question}], tokenize=False, add_generation_prompt=True)
print(prompt)
input_ids = tok(prompt, return_tensors="pt").input_ids.to("cuda")

output = model.generate(input_ids, max_new_tokens=32, eos_token_id=tok.eos_token_id)
print(tok.decode(output[0]))

## 6. Deploy the LLM for Production

You can now deploy your model to production. For deploying open LLMs into production we recommend using [Text Generation Inference (TGI)](https://github.com/huggingface/text-generation-inference). TGI is a purpose-built solution for deploying and serving Large Language Models (LLMs). TGI enables high-performance text generation using Tensor Parallelism and continous batching for the most popular open LLMs, including Llama, Mistral, Mixtral, StarCoder, T5 and more. Text Generation Inference is used by companies as IBM, Grammarly, Uber, Deutsche Telekom, and many more. There are several ways to deploy your model, including:

 * [Deploy LLMs with Hugging Face Inference Endpoints](https://huggingface.co/blog/inference-endpoints-llm)
 * [Hugging Face LLM Inference Container for Amazon SageMaker](https://huggingface.co/blog/sagemaker-huggingface-llm)

If you have docker installed you can use the following command to start the inference server. 

_Note: Make sure that you have enough GPU memory to run the container. Restart kernel to remove all allocated GPU memory from the notebook._ 

In [ ]:
docker run -d --name tgi --gpus all -ti -p 8080:80 \
%%bash 
# model=$PWD/{args.output_dir} # path to model
model=$PWD/data/llama-3-1-8b-math-orca-qlora # path to model
num_shard=1             # number of shards
max_input_length=1024   # max input length
max_total_tokens=2048   # max total tokens


docker run --gpus all -ti -p 8080:80 \
  -e MODEL_ID=/workspace \
  -e NUM_SHARD=$num_shard \
  -e MAX_INPUT_LENGTH=$max_input_length \
  -e MAX_TOTAL_TOKENS=$max_total_tokens \
  -e HF_TOKEN=$(cat ~/.cache/huggingface/token) \
  -v $model:/workspace \
  ghcr.io/huggingface/text-generation-inference:2.4.0

In [ ]:
model=data/llama-3-1-8b-math-orca-qlora # path to model
num_shard=1             # number of shards
max_input_length=1024   # max input length
max_total_tokens=2048   # max total tokens

docker run --gpus 2 -ti -p 8080:80 -e HF_TOKEN=$(cat ~/.cache/huggingface/token) ghcr.io/huggingface/text-generation-inference:2.4.1 --model-id philschmid/llama-3-1-8b-math-orca-qlora --num-shard 2

docker run --runtime nvidia --gpus all \
    -p 8000:8000 \
    --env "HUGGING_FACE_HUB_TOKEN=$(cat ~/.cache/huggingface/token)" \
    vllm/vllm-openai --model Meta-Llama/Meta-Llama-3.1-8B \
    --enable-lora --lora-modules  orca=philschmid/llama-3-1-8b-math-orca-qlora \
    --max-model-len 2048

Once your container is running you can send requests using the `openai` or `huggingface_hub` sdk. Here we ll use the `openai` sdk to send a request to our inference server. If you don't have the `openai` sdk installed you can install it using `pip install openai`.

In [3]:
from openai import OpenAI
from datasets import load_dataset
from random import randint

# create client 
client = OpenAI(base_url="http://localhost:8000/v1",api_key="-")

system_message = """Solve the given high school math problem by providing a clear explanation of each step leading to the final solution.

Provide a detailed breakdown of your calculations, beginning with an explanation of the problem and describing how you derive each formula, value, or conclusion. Use logical steps that build upon one another, to arrive at the final answer in a systematic manner.

# Steps

1. **Understand the Problem**: Restate the given math problem and clearly identify the main question and any important given values.
2. **Set Up**: Identify the key formulas or concepts that could help solve the problem (e.g., algebraic manipulation, geometry formulas, trigonometric identities).
3. **Solve Step-by-Step**: Iteratively progress through each step of the math problem, justifying why each consecutive operation brings you closer to the solution.
4. **Double Check**: If applicable, double check the work for accuracy and sense, and mention potential alternative approaches if any.
5. **Final Answer**: Provide the numerical or algebraic solution clearly, accompanied by appropriate units if relevant.

# Notes

- Always clearly define any variable or term used.
- Wherever applicable, include unit conversions or context to explain why each formula or step has been chosen.
- Assume the level of mathematics is suitable for high school, and avoid overly advanced math techniques unless they are common at that level.
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?"},
]
expected_answer = "72"

# Take a random sample from the dataset and remove the last message and send it to the model
response = client.chat.completions.create(
	model="orca",
	messages=messages,
	stream=False, # no streaming
	max_tokens=256,
)
response = response.choices[0].message.content

# Print results
print(f"Query:\n{messages[1]['content']}")
print(f"Original Answer:\n{expected_answer}")
print(f"Generated Answer:\n{response}")

APIConnectionError: Connection error.

Awesome, Don't forget to stop your container once you are done.

In [ ]:
!docker stop tgi

# Appendix 

## Distributed Training

### Deepspeed + Q-LoRA

Note: change the `num_processes` to the number of GPUs you want to use.

```bash
accelerate launch --config_file configs/accelerate_configs/deepspeed_zero3.yaml --num_processes 4 scripts/run_sft.py --config receipes/llama-3-1-8b-qlora.yaml
```